<a href="https://colab.research.google.com/github/victen18/distilBERT/blob/main/Sentiment_Classification_using_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [4]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)

In [5]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [6]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [7]:
data_train.shape,data_test.shape

((25000, 2), (25000, 2))

In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [9]:
(train,val,preproc) = text.texts_from_df(train_df=data_train,text_column='Reviews',label_columns='Sentiment',val_df=data_test,maxlen=512,preprocess_mode='distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [10]:
model = text.text_classifier(name='distilbert',train_data=train,preproc=preproc,metrics=['accuracy'])

Is Multi-Label? False
maxlen is 512
done.


In [11]:
learner =  ktrain.get_learner(model=model,train_data=train,val_data=val,batch_size=6,use_multiprocessing=True)

In [12]:
learner.fit_onecycle(lr=2e-5,epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - 2181s 523ms/step - loss: 0.2903 - accuracy: 0.8784 - val_loss: 0.1506 - val_accuracy: 0.9488
Epoch 2/2
4167/4167 [==============================] - 2192s 526ms/step - loss: 0.1525 - accuracy: 0.9444 - val_loss: 0.0590 - val_accuracy: 0.9845


In [13]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
predictor.save('/content/drive/MyDrive/distilBERT')

In [16]:
data = ['this moves are really bad. acting was also bad. I will not watch again',
        'the movie was really great, i will watch it again','Another great movie to watch with family']

In [17]:
predictor.predict(data)

['neg', 'pos', 'pos']

In [18]:
predictor.get_classes()

['neg', 'pos']

In [19]:
predictor.predict(data,return_proba=True)

array([[0.9947025 , 0.00529746],
       [0.00420365, 0.9957963 ],
       [0.00605527, 0.9939447 ]], dtype=float32)